In [1]:
#check librosa stft and istft
import numpy as np

from dataio import readdata, readlabels, writedata
from datafilters import apply_dc_filter, apply_dwt_filter, apply_stfft_filter

In [2]:
#Hyperparams

#DC Filter
enable_dc = True
dc_lowcut = 1.0
dc_highcut = 13.0
dc_order = 2
dc_type = "bandpass"
dc_func_type = "butter"

#DWT Filter
enable_dwt = False
dwt_type = "db2"
dwt_level = 4
dwt_thresh_func = "soft"
dwt_thresh_type = "rigrsure"

#STFT Filter
enable_fft = False
fft_window = 0.50
fft_step = 0.25
fft_thresh = 2.0
fft_set_thresh = 0.0

In [3]:
predataset = readdata("./curated/raw-presamples")
dataset = readdata("./curated/raw-samples")
#Constants
fs = 250.0 #Frequency in Hz
sample_time = dataset.shape[1]/fs #Total time for sample
presample_time = predataset.shape[1]/fs #Total time for sample

#To get alpha and beta waves
dataset.flags['WRITEABLE'] = True
predataset.flags['WRITEABLE'] = True

y_s = np.copy(dataset[0,:,0])
y_p = np.copy(predataset[0,:,0])

for i in range(0,dataset.shape[0]):
    for j in range(0,dataset.shape[2]):
        if enable_dc:
            predataset[i,:,j] = apply_dc_filter(predataset[i,:,j], fs, dc_lowcut, dc_highcut, dc_order, dc_type, dc_func_type)
            dataset[i,:,j] = apply_dc_filter(dataset[i,:,j], fs, dc_lowcut, dc_highcut, dc_order, dc_type, dc_func_type)
        if enable_dwt:
            predataset[i,:,j] = apply_dwt_filter(predataset[i,:,j], dwt_type, dwt_level, dwt_thresh_func, dwt_thresh_type)
            dataset[i,:,j] = apply_dwt_filter(dataset[i,:,j], dwt_type, dwt_level, dwt_thresh_func, dwt_thresh_type)
        if enable_fft:
            predataset[i,:,j] = apply_stfft_filter(predataset[i,:,j], fs, presample_time, fft_window, fft_step, fft_thresh, fft_set_thresh)
            dataset[i,:,j] = apply_stfft_filter(dataset[i,:,j], fs, sample_time, fft_window, fft_step, fft_thresh, fft_set_thresh)
        #Normalize
        dataset[i,:,j] = dataset[i,:,j]/np.linalg.norm(dataset[i,:,j])
        #predataset[i,:,j] = predataset[i,:,j]/np.linalg.norm(predataset[i,:,j])
        #dataset[i,:,j] = dataset[i,:,j]/np.linalg.norm(dataset[i,:,j])
        #- np.mean(predataset[i,:,j])

In [4]:
test_size = 0.15
valid_size = 0.15
labels = readlabels("./curated/raw-inputs")

# Test Set
dataindex = range(0,dataset.shape[0])
test_index = np.random.choice(dataindex, int(dataset.shape[0]*test_size))
test_dataset = dataset[test_index,:,:]
test_labels = labels[test_index]
dataset = np.delete(dataset, test_index, axis = 0)
labels = np.delete(labels, test_index, axis = 0)

# Validation Set
#dataindex = range(0,dataset.shape[0])
#valid_index = np.random.choice(dataindex, int(dataset.shape[0]*valid_size))
#valid_dataset = dataset[valid_index,:,:]
#valid_labels = labels[valid_index]
#dataset = np.delete(dataset, valid_index, axis = 0)
#labels = np.delete(labels, valid_index, axis = 0)

# Train Set
train_dataset = dataset
train_labels = labels

print('Training:', train_dataset.shape, train_labels.shape)
#print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Training: (1106, 470, 8) (1106,)
Testing: (193, 470, 8) (193,)


In [5]:
#Write training
writedata("./curated/train_dataset", train_dataset)
writedata("./curated/train_labels", train_labels)
#Write validation
#writedata("./curated/valid_dataset", valid_dataset)
#writedata("./curated/valid_labels", valid_labels)
#Write test
writedata("./curated/test_dataset", test_dataset)
writedata("./curated/test_labels", test_labels)
#plt.figure(figsize=(15,8))
#plt.plot(dataset[0,:,0])
#plt.show()

In [6]:
#Read training
train_dataset = readdata("./curated/train_dataset")
train_labels = readlabels("./curated/train_labels")
#Read validation
#valid_dataset = readdata("./curated/valid_dataset")
#valid_labels = readlabels("./curated/valid_labels")
#Read test
test_dataset = readdata("./curated/test_dataset")
test_labels = readlabels("./curated/test_labels")

print('Training:', train_dataset.shape, train_labels.shape)
#print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Training: (1106, 470, 8) (1106,)
Testing: (193, 470, 8) (193,)


In [ ]:
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
y_temp = y_p
sample_time_temp = presample_time

#Plots for comparison
plt.figure(1);
plt.figure(figsize=(10,12));

#Raw EEG Signal
plt.subplot(511);
plt.plot(y_temp);
plt.title("Raw EEG")

#DC Filter
y_temp1 = apply_dc_filter(y_temp, fs, dc_lowcut, dc_highcut, dc_order, dc_type, dc_func_type)
plt.subplot(512);
plt.plot(y_temp1);
plt.title("EEG + DC Filter");

#Discrete Wavelet Transform
y_temp2 = apply_dwt_filter(y_temp1, dwt_type, dwt_level, dwt_thresh_func, dwt_thresh_type)
plt.subplot(513);
plt.plot(y_temp2);
plt.title("EEG + DC + Discrete Wavelet Transform");

#Fast Fourier Transform
#y_temp3 = apply_stfft_filter(y_temp2, fs, sample_time_temp, fft_window, fft_step, fft_thresh, fft_set_thresh)
#plt.subplot(514);
#plt.plot(y_temp3);
#plt.title("EEG + DC + DWT + Fast Fourier Transform");


#Current
plt.subplot(515);
plt.plot(dataset[0,:,0]);
plt.title("Current");
#plt.show()

In [ ]:
#Plots for comparison
plt.figure(1);
plt.figure(figsize=(10,12));

#Presample signal
plt.subplot(311);
plt.plot(predataset[20,:,2]);
plt.title("Presample")

#Sample signal
plt.subplot(312);
plt.plot(dataset[20,:,2]);
plt.title("Sample");

sam_mean = np.mean(predataset[20,:,2])
dataset[20,:,2] = dataset[20,:,2] - sam_mean

#Sample signal
plt.subplot(312);
plt.plot(dataset[20,:,2]);
plt.title("Sample");
print(sam_mean)